In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Input and output file paths
input_csv = "profiles.csv"     # contains profile URLs in one column
output_csv = "listings.csv"    # output file with scraped links

# Read profile URLs from CSV (assuming first column has the URLs)
profiles = pd.read_csv(input_csv, header=None)[0].tolist()

base_url = "https://www.spareroom.co.uk"
headers = {"User-Agent": "Mozilla/5.0"}

all_results = []

for profile_url in profiles:
    try:
        response = requests.get(profile_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract links from listing cards
        for a in soup.find_all("a", class_="listing-card__link"):
            href = a.get("href")
            if href:
                full_url = href if href.startswith("http") else base_url + href
                all_results.append({"profile": profile_url, "listing": full_url})
    except Exception as e:
        print(f"Error fetching {profile_url}: {e}")

# Save results to CSV
df = pd.DataFrame(all_results)
df.to_csv(output_csv, index=False)

print(f"Scraped {len(all_results)} listings and saved to {output_csv}")


Scraped 161 listings and saved to listings.csv


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import re
import time
import json

def clean_text(text):
    """Clean and normalize text data (for titles, small fields)"""
    if not text or text == "N/A":
        return None
    text = re.sub(r'\s+', ' ', text.strip())  # remove extra whitespace
    text = re.sub(r'<[^>]+>', '', text)       # remove HTML tags
    if len(text) > 500:
        text = text[:500] + "..."
    return text

def extract_rich_text(elem):
    """Extract text while preserving emojis and newlines from <br> tags (for description only)"""
    if not elem:
        return None
    for br in elem.find_all("br"):
        br.replace_with("\n")
    text = elem.get_text()
    text = text.strip()
    return text if text else None

def extract_price(price_text):
    """Extract and normalize price information (convert pw → monthly if needed)"""
    if not price_text or price_text == "N/A":
        return None

    price_match = re.search(r'£?(\d+(?:,\d+)?(?:\.\d{2})?)', str(price_text))
    if price_match:
        price = price_match.group(1).replace(',', '')
        try:
            price = float(price)
            text_lower = price_text.lower()

            if "pw" in text_lower or "per week" in text_lower:
                # Convert weekly → monthly (average 52 weeks / 12 months)
                price = round(price * 52 / 12)

            else:
                # Assume price is already per month
                price = round(price)

            return int(price)
        except:
            return None
    return None


def extract_coordinates(lat_text, lon_text):
    """Extract and validate coordinates"""
    try:
        lat = float(lat_text) if lat_text and lat_text != "N/A" else None
        lon = float(lon_text) if lon_text and lon_text != "N/A" else None
        if lat is not None and (-90 <= lat <= 90):
            lat = round(lat, 8)
        else:
            lat = None
        if lon is not None and (-180 <= lon <= 180):
            lon = round(lon, 8)
        else:
            lon = None
        return lat, lon
    except:
        return None, None

def extract_feature_list(soup):
    """Extracts key-value pairs from feature-list <dl> blocks"""
    features = {}
    for dl in soup.find_all("dl", class_="feature-list"):
        keys = dl.find_all("dt", class_="feature-list__key")
        vals = dl.find_all("dd", class_="feature-list__value")

        for k, v in zip(keys, vals):
            key = clean_text(k.get_text()) if k else None
            val = clean_text(v.get_text()) if v else None

            # Handle tick/cross spans ("Yes"/"No")
            if v and v.find("span", class_="tick"):
                val = "Yes"
            elif v and v.find("span", class_="cross"):
                val = "No"

            if key:
                features[key] = val
    return features

def scrape_listing_advanced(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        }
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # Title
        title = "N/A"
        title_elem = soup.find("h1") or soup.find("h2") or soup.find("title")
        if title_elem:
            title = clean_text(title_elem.get_text())

        # Agent / Landlord name
        agent_name = None
        agent_elem = soup.find("strong", class_="profile-photo__name")
        if agent_elem:
            agent_name = clean_text(agent_elem.get_text())

       # Extract location (always the 2nd <li> inside .key-features)
        location = None
        key_features = soup.find("ul", class_="key-features")
        if key_features:
         items = key_features.find_all("li", class_="key-features__feature")
         if len(items) >= 2:
          location = items[1].get_text(strip=True)  # ✅ "Devons Road"


        # Latitude / Longitude
        latitude, longitude = "N/A", "N/A"
        script_tags = soup.find_all("script")
        for script in script_tags:
            if script.string:
                script_content = script.string
                lat_match = re.search(r'latitude["\s]*:["\s]*"?([0-9.-]+)"?', script_content)
                lon_match = re.search(r'longitude["\s]*:["\s]*"?([0-9.-]+)"?', script_content)
                if lat_match:
                    latitude = lat_match.group(1)
                if lon_match:
                    longitude = lon_match.group(1)
                location_match = re.search(
                    r'location["\s]*:["\s]*{[^}]*latitude["\s]*:["\s]*"?([0-9.-]+)"?[^}]*longitude["\s]*:["\s]*"?([0-9.-]+)"?',
                    script_content
                )
                if location_match:
                    latitude, longitude = location_match.group(1), location_match.group(2)

        # Photos (only inside photo-gallery containers)
        all_photo_urls = []

        # ✅ Main image container
        main_gallery = soup.select_one("dl.photo-gallery__main-image-wrapper")
        if main_gallery:
            main_links = main_gallery.find_all("a", href=re.compile(
                r"^https://photos2\.spareroom\.co\.uk/images/flatshare/listings/large/[0-9]+/[0-9]+/[0-9]+\.jpg$"
            ))
            for link in main_links:
                photo_url = link.get("href")
                if photo_url and photo_url not in all_photo_urls:
                    all_photo_urls.append(photo_url)

        # ✅ Thumbnail gallery container
        thumb_gallery = soup.select_one("div.photo-gallery__thumbnails")
        if thumb_gallery:
            thumb_links = thumb_gallery.find_all("a", href=re.compile(
                r"^https://photos2\.spareroom\.co\.uk/images/flatshare/listings/large/[0-9]+/[0-9]+/[0-9]+\.jpg$"
            ))
            for link in thumb_links:
                photo_url = link.get("href")
                if photo_url and photo_url not in all_photo_urls:
                    all_photo_urls.append(photo_url)

        first_photo_url = all_photo_urls[0] if all_photo_urls else None
        photo_count = len(all_photo_urls)
        all_photos = ", ".join(all_photo_urls) if all_photo_urls else None



        # Price
        price = None
        price_selectors = [".price", ".rent", ".amount", "[class*='price']", "[class*='rent']", "[class*='amount']"]
        for selector in price_selectors:
            try:
                elements = soup.select(selector)
                for elem in elements:
                    text = elem.get_text(strip=True)
                    if '£' in text:
                        price = extract_price(text)
                        if price:
                            break
            except:
                continue

        # ✅ Description (preserve emojis + newlines)
        description = None
        detaildesc_elem = soup.find("p", class_="detaildesc")
        if detaildesc_elem:
            description = extract_rich_text(detaildesc_elem)
        else:
            feature_desc_body = soup.find("div", class_="feature__description-body")
            if feature_desc_body:
                description = extract_rich_text(feature_desc_body)
            else:
                desc_selectors = [
                    ".description", ".details", ".content",
                    "[class*='description']", "[class*='details']",
                    "[class*='content']", "p", ".listing-details"
                ]
                for selector in desc_selectors:
                    try:
                        elements = soup.select(selector)
                        for elem in elements:
                            text = extract_rich_text(elem)
                            if text and len(text) > 50:
                                description = text
                                break
                        if description:
                            break
                    except:
                        continue

        # Property type
        property_type = None
        type_keywords = ["room", "bedroom", "studio", "flat", "apartment", "house", "en-suite", "ensuite"]
        search_text = (str(title) + " " + str(description)).lower()
        for keyword in type_keywords:
            if keyword in search_text:
                property_type = keyword.title()
                break

        # ✅ Extract structured features
        features = extract_feature_list(soup)
        available_date = features.get("Available")
        min_term = features.get("Minimum term")
        max_term = features.get("Maximum term")
        deposit = features.get("Deposit")
        bills_included = features.get("Bills included?")
        furnishings = features.get("Furnishings")
        parking = features.get("Parking")
        garden = features.get("Garden/patio")
        broadband = features.get("Broadband included")
        housemates = features.get("# housemates")
        total_rooms = features.get("Total # rooms")
        smoker = features.get("Smoker?")
        pets = features.get("Any pets?")
        occupation = features.get("Occupation")
        gender = features.get("Gender")
        couples_ok = features.get("Couples OK?")
        smoking_ok = features.get("Smoking OK?")
        pets_ok = features.get("Pets OK?")
        pref_occupation = features.get("Occupation")  # new housemate occupation
        references = features.get("References?")
        min_age = features.get("Min age")
        max_age = features.get("Max age")

        # Final result dictionary
        lat, lon = extract_coordinates(latitude, longitude)
        result = {
            "url": url,
            "title": title,
            "agent_name": agent_name,
            "location": location,
            "latitude": lat,
            "longitude": lon,
            "status": "available",
            "price": price,
            "description": description,  # ✅ emojis + newlines preserved
            "property_type": property_type,
            "available_date": available_date,
            "min_term": min_term,
            "max_term": max_term,
            "deposit": deposit,
            "bills_included": bills_included,
            "furnishings": furnishings,
            "parking": parking,
            "garden": garden,
            "broadband": broadband,
            "housemates": housemates,
            "total_rooms": total_rooms,
            "smoker": smoker,
            "pets": pets,
            "occupation": occupation,
            "gender": gender,
            "couples_ok": couples_ok,
            "smoking_ok": smoking_ok,
            "pets_ok": pets_ok,
            "pref_occupation": pref_occupation,
            "references": references,
            "min_age": min_age,
            "max_age": max_age,
            "photo_count": photo_count,
            "first_photo_url": first_photo_url,
            "all_photos": all_photos,
            "photos": json.dumps(all_photo_urls) if all_photo_urls else None,
        }

        return result
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

def open_csv(filename):
    """Try UTF-8 first, fallback to latin-1"""
    try:
        return open(filename, newline="", encoding="utf-8")
    except UnicodeDecodeError:
        return open(filename, newline="", encoding="latin-1")

def main():
    results = []
    with open("listings.csv", newline="", encoding="utf-8", errors="replace") as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) < 2:  # skip rows without a second column
                continue
            url = row[1].strip()   # ✅ use second column instead of first
            if not url:
                continue
            print(f"Scraping {url}")
            data = scrape_listing_advanced(url)
            if data:
                results.append(data)
            time.sleep(1)  # be polite

    # Columns to save
    database_columns = [
        "url", "title", "agent_name", "location", "latitude", "longitude", "status", "price",
        "description", "property_type", "available_date", "min_term", "max_term",
        "deposit", "bills_included", "furnishings", "parking", "garden",
        "broadband", "housemates", "total_rooms", "smoker", "pets", "occupation",
        "gender", "couples_ok", "smoking_ok", "pets_ok", "pref_occupation",
        "references", "min_age", "max_age", "photo_count", "first_photo_url",
        "all_photos", "photos"
    ]

    output_filename = "newscrape.csv"
    with open(output_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=database_columns)
        writer.writeheader()
        writer.writerows(results)

    print(f"Scraping complete. Results saved to {output_filename}")
    print(f"Successfully scraped {len(results)} properties from all_vac.csv")

if __name__ == "__main__":
    main()


Scraping listing
Failed to scrape listing: Invalid URL 'listing': No scheme supplied. Perhaps you meant https://listing?
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id=16801986&search_id=&city_id=&flatshare_type=offered&search_results=%2Fu2901752&
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id=17984292&search_id=&city_id=&flatshare_type=offered&search_results=%2Fu2901752&
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id=17999696&search_id=&city_id=&flatshare_type=offered&search_results=%2Fu2901752&
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id=15708796&search_id=&city_id=&flatshare_type=offered&search_results=%2Fu2901752&
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id=16767096&search_id=&city_id=&flatshare_type=offered&search_results=%2Fu2901752&
Scraping https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?flatshare_id